In [86]:
import mpi4py
import pyomo.environ as pyo
import mpisppy.utils.sputils as sputils
import numpy as np
import pandas as pd

In [147]:
def build_model(c_pv, c_wind, c_storage, c_in, c_out, timesteps, time_scale, storage_scale, pv, wind, demand):
    model = pyo.ConcreteModel()

    # Variables
    model.u_pv = pyo.Var(within=pyo.NonNegativeReals)
    model.u_wind = pyo.Var(within=pyo.NonNegativeReals)
    model.u_storage = pyo.Var(within=pyo.NonNegativeReals)
    model.grid_out = pyo.Var([t for t in timesteps], within=pyo.NonNegativeReals)
    model.grid_in = pyo.Var([t for t in timesteps], within=pyo.NonNegativeReals)
    model.storage = pyo.Var([t for t in timesteps], within=pyo.Reals)
    model.buyback = pyo.Var([t for t in timesteps], within=pyo.Reals)

    # Objectives
    model.INVESTMENT_COST = model.u_pv*c_pv+model.u_wind*c_wind+model.u_storage*c_storage
    model.OPERATION_COST = sum(model.grid_in)*c_in - sum(model.grid_out)*c_out
    model.OBJ = pyo.Objective(
        expr=model.INVESTMENT_COST/time_scale + model.OPERATION_COST,
        sense=pyo.minimize)
    
    # Constraints
    model.CONSTR= pyo.ConstraintList()
    for t in timesteps:
        model.CONSTR.add(pyo.summation(model.storage, index = range(t))>=-0.5*model.u_storage*storage_scale)
        model.CONSTR.add(pyo.summation(model.storage, index = range(t))<=0.5*model.u_storage*storage_scale)
        model.CONSTR.add(model.grid_in[t] - model.grid_out[t] + model.u_pv*pv[t] + model.u_wind*wind[t] - demand[t] + model.storage[t] + model.buyback[t] == 0)

    #def energy_balance_rule(model, demand):
    #    return model.grid_in - model.grid_out + pv*model.u_pv + wind*model.u_wind - demand + model.storage + model.buyback == 0
    #def storage_capacity_rule(model, storage_scale):
    #    return (-0.5*model.u_storage*storage_scale, sum(model.storage), 0.5*model.u_storage*storage_scale)
    #model.ENERGY_BALANCE = pyo.Constraint(rule = energy_balance_rule(model, demand))
    #model.STORAGE_CAPACITY = pyo.Constraint(rule = storage_capacity_rule(model, storage_scale))
    #
    return model

In [148]:
c_i = .03
c_o = .01
timesteps = range(24)
time_scale = 10. *365*24/len(timesteps)
c_pv = 1000.
c_wind = 1000.
c_storage = 400.
c_flex = .5
F = 3
storage_scale = .5

pv_data = pd.read_csv("../data/pv_Halle18.csv")
wind_data = pd.read_csv("../data/wind_Karholz.csv")
demand_data = pd.read_csv("../data/demand_Industriepark.csv")

In [149]:
pv_data = np.genfromtxt("../data/pv_Halle18.csv", delimiter=",")[1:(len(timesteps)+1)]
wind_data = np.genfromtxt("../data/wind_Karholz.csv", delimiter=",")[1:(len(timesteps)+1)]
demand_data = np.genfromtxt("../data/demand_Industriepark.csv", delimiter=",")[1:(len(timesteps)+1)]

In [150]:
model = build_model(c_pv, c_wind, c_storage, c_i, c_o, timesteps, time_scale, storage_scale, pv_data, wind_data, demand_data)

In [151]:
solver = pyo.SolverFactory("cbc")
solver.solve(model)
print(f"{pyo.value(model.OBJ):.1f}")

5.5


In [153]:
pyo.value(model.u_wind)

0.0

In [116]:
sputils.attach_root_node(model, model.INVESTMENT_COST, model.storage)